##랭체인을 이용한 GPT-4호출하기
랭체인을 이용해서 GPT-4를 호출하는 방법과 이전 대화를 기억하는 방법에 대해 다룹니다.이제 필요한 랭체인 도구들을 임포트 합니다.

In [ ]:
!pip install langchain_community langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.tracers.stdout import ConsoleCallbackHandler

다음으로 os.environ['OPENAI_API_KEY']에 발급받은 OpenAI API 키 값을 입력합니다. 해당 코
드는 현재 실습 환경에 OpenAI API 키 값을 세팅한다는 의미입니다.

In [ ]:
os.environ['OPENAI_API_KEY'] = "Openai_api_key"

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,   # 창의성 (0.0~2.0)
    max_tokens=2048,
    model_name = "gpt-4o"
)

랭체인 패키지를 통해서도 오픈 AI 의 GPT‐4 API 를 사용할 수 있는데, openai 패키지를 통해 GPT‐4 API 를 사용하는 방법과 랭체인 패키지를 통해 GPT‐4 API 를 사용하는 방법은 다릅니다. 랭체인 패키지를 사용해 GPT‐4 를 사용하기 위해서는 ChatOpenAI()를 이용하여 llm 객체를 생성해야 합니다. 이때 openai 패키지로 GPT‐4 API 를 사용할 때와 마찬가지로 ChatOpenAI() 내부에 다양한 설정값들을 세팅할 수 있습니다.

예를 들어, temperature는 GPT‐4 에게 무작위성 또는 창의성을 주는 파라미터로서 0~2 사이의 값을
가지며, 0 에 가까울수록 LLM 은 사실적인 답변을 내놓고, 2 에 가까울수록 의외의 답변을 내놓습니다. max_tokens는 GPT‐4 의 답변의 길이를 조절하는 값이며, model_name은 다양한 GPT‐4 모델 중 사용하고자 하는 모델의 이름을 설정하는 곳입니다. 이 파라미터들은 랭체인이 아니라 openai 패키지를 통해 GPT‐4 를 사용할 때도 사용할 수 있는 동일한 파라미터들입니다.

이렇게 선언된 llm 객체는 invoke()를 통해 사용자의 질문을 전달하고 답변을 얻을 수 있습니다. 다음 코드에서는 "세종대왕이 누구인지 설명해주세요"라는 질문을 던지고 답변을 출력합니다. invoke()를 통해 얻은 답변을 result에 저장하고, 답변을 출력하기 위해서는 .content를 result 뒤에 붙여서 출력합니다.

In [ ]:
# 질의 내용
question = '세종대왕이 누구인지 설명해주세요'

# 질의
result = llm.invoke(question)
print(result.content)

세종대왕은 조선의 제4대 왕으로, 한국 역사에서 가장 존경받는 군주 중 한 명입니다. 그는 1397년에 태어나 1418년부터 1450년까지 재위하였습니다. 세종대왕은 여러 방면에서 뛰어난 업적을 남겼는데, 그 중 가장 유명한 것은 한글의 창제입니다. 한글은 백성들이 쉽게 읽고 쓸 수 있도록 고안된 문자 체계로, 1443년에 창제되어 1446년에 반포되었습니다.

세종대왕은 또한 과학, 기술, 문화, 경제 등 다양한 분야에서 발전을 이끌었습니다. 그는 천문학, 의학, 농업 등 여러 분야의 연구를 장려하고, 측우기와 같은 과학 기구를 개발하도록 지원하였습니다. 또한, 음악과 예술을 장려하고, 법률과 제도를 정비하여 국가의 기틀을 다졌습니다. 세종대왕의 통치 아래 조선은 정치적 안정과 문화적 번영을 누릴 수 있었습니다.


GPT‐4 에게 좀 더 복잡한 프롬프트를 전달하기 위해서 랭체인에서는 프롬프트 템플릿이라는 것을 사용합니다. 프롬프트 템플릿이란 말 그대로 GPT‐4 에게 입력을 전달하는 프롬프트의 템플릿을 만들어 두는 것을 의미합니다. 프롬프트 템플릿에서는 일종의 변수 (variables) 를 만들어두고, 정해진 템플릿 내에서 변수만 변경하여 GPT‐4 에게 입력을 전달하는 것이 가능합니다. 예를 들어, {who}라는 변수를
두고, "{who}가 누구인지 설명해주세요"라는 템플릿을 만든다고 해봅시다. 이제 해당 템플릿에서 {who}라는 변수만 계속 변경하여 GPT‐4 에게 입력을 전달할 수 있습니다.
다음은 프롬프트 템플릿을 만드는 과정입니다. 먼저 {} 로 감싼 변수를 두고, 템플릿을 만들어 template 이 라는 문자열에 저장한 뒤, PromptTemplate() 에 template 의 인자값으로 전달합니다. 그리고 사용할 변수명은 input_variables 의 값으로 전달합니다.

In [ ]:
# 질문 템플릿 형식 정의
template = "{who}가 누구인지 설명해주세요"

# 템플릿 완성
prompt = PromptTemplate(
    template = template, input_variables = ["who"]
)
print(prompt)

input_variables=['who'] input_types={} partial_variables={} template='{who}가 누구인지 설명해주세요'


이렇게 만들어진 프롬프트 템플릿에 .format()을 명시적으로 추가하면 프롬프트 완성본을 미리 볼 수 있습니다.

In [ ]:
print(prompt.format(who="이순신"))

이순신가 누구인지 설명해주세요


이렇게 만들어진 프롬프트 템플릿은 ChatOpenAI()를 통해 생성한 llm 객체와 연결할 수 있습니다.
직접 GPT‐4 를 호출하는 llm 객체와 프롬프트 템플릿을 연결하는 매개체를 랭체인에서는 개념적으로 체인 (Chain) 이라고 부릅니다. 체인을 생성하는 방법은 간단한데, 프롬프트 템플릿과 llm 객체 사이에 prompt | llm과 같이 |(파이프) 를 추가하는 것입니다. 이는 프롬프트를 모델에 전달한다는 의미를 담고 있습니다. 이렇게 생성된 llm_chain 객체는 앞서 llm 객체와 마찬가지로 invoke()를 통해 GPT‐4 에게 입력을 전달할 수 있습니다. 이번에는 프롬프트 템플릿을 사용하고 있으므로 프롬프트 템플릿의 변수 값도 전달해 줍니다. 여기서는 who의 값으로 "이순신 장군"을 전달했습니다. 프롬프트 템플릿이 "{who}가 누구인지 설명해주세요"라는 템플릿을 갖고 있으므로 결과적으로 GPT‐4 에는 "이순신 장군가 누구인지 설명해주세요"라는 입력이 전달됩니다.

In [ ]:
# 연결된 체인 생성
llm_chain = prompt | llm
result = llm_chain.invoke({"who": "이순신 장군"})
print(result.content)

이순신 장군은 조선 시대의 유명한 군인으로, 임진왜란 당시 조선 수군을 이끌어 일본군을 상대로 여러 차례 승리를 거둔 인물입니다. 그는 1545년에 태어나 1598년에 전사할 때까지 조선의 해군을 지휘하며 뛰어난 전략과 전술로 유명했습니다. 특히, 한산도 대첩, 명량 해전 등에서의 승리는 그의 지휘 능력을 잘 보여줍니다.

이순신 장군은 거북선이라는 철갑선을 활용하여 일본군을 효과적으로 물리쳤으며, 그의 리더십과 용기는 오늘날까지도 많은 사람들에게 존경받고 있습니다. 그는 충무공이라는 시호를 받았으며, 한국 역사에서 가장 위대한 군사 지도자 중 한 명으로 평가받고 있습니다.


llm_chain으로 invoke()를 실행할 때 실제 전달되는 프롬프트 또한 출력 결과로 확인하고 싶다면 invoke()를 호출할 때 config={'callbacks': [ConsoleCallbackHandler()]}를 사
용하면 됩니다.

In [ ]:
result = llm_chain.invoke({"who": "이순신 장군"},
                          config={"callbacks": [ConsoleCallbackHandler()]})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "who": "이순신 장군"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "who": "이순신 장군"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 이순신 장군가 누구인지 설명해주세요"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [3.60s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "이순신 장군은 조선 시대의 유명한 군인으로, 임진왜란 당시 조선 수군을 이끌며 큰 공을 세운 인물입니다. 1545년에 태어나 1598년에 전사한 그는, 특히 한산도 대첩, 명량 해전, 노량 해전 등 여러 해전에서 혁혁한 전과를 올렸습니다. 이순신 장군은 그의 뛰어난 전략과 전술, 그리고 리더십으로 인해 조선의 해군력을 크게 강화하였고, 일본군의 침략을 효과적으로 막아냈습니다.\n\n그는 또한 거북선이라는 철갑선을 사용하여 해전에서 큰 이점을 얻었으며, 이는 세계 해전사에서도 주목받는 혁신적인 무기였습니다. 이순신 장군은 그의 충성과 용맹함으로 인해 한국 역사에서 가장 존경받는 인물 중 하나로 남아 있으며, 그의 삶과 업적은 오늘날까지도 많은 사람들에게 영감을 주고 있습니다.",
        "generation_i

답변을 얻기까지의 다양한 과정을 출력하는데 목적이 있는 코드이기 때문에 다양한 정보가 출력되는데,그중 “prompts” 에 해당하는 부분이 실제 GPT‐4 에 전달된 프롬프트이며, 여기서는 “Human: 이순신 장군가 누구인지 설명해주세요” 라는 프롬프트가 전달되었습니다.

이번에는 과거 대화 내역을 반영하여 GPT‐4 와 대화할 수 있는 RunnableWithMessageHistory()
에 대해 알아봅시다. 우선 history와 input이라는 두 개의 변수를 가지는 새로운 프롬프트 템플릿을
선언합니다. 이때 history는 과거의 대화 내역을 지속적으로 누적할 변수입니다. input은 사용자
의 최근 입력이 들어갈 변수입니다. 다시 말해 사용자가 GPT‐4 와 대화를 하면 할수록 사용자의 입력과 GPT‐4 의 답변을 포함한 모든 기록은 아래 템플릿에서 {history}에 누적되며, 현재의 입력하는 질문은 {input}에 들어가서 완성된 프롬프트가 GPT‐4 에게 전달됩니다. GPT‐4 는 이러한 프롬프트를 통해서 현재의 질문과 이전의 모든 대화 내역을 바탕으로 답변할 수 있습니다.

In [ ]:
# 프롬프트 템플릿 생성
template = """아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 위키독스봇 입니다.
대화 문맥을 바탕으로 친절한 답변을 진행하세요.

Current Conversation:
{history}

Human: {input}
AI:"""

prompt = PromptTemplate(
    template=template, input_variables=['history', 'input']
)

# llm 객체를 새로 선언하고, 프롬프트 템플릿과 llm 객체를 연결합니다.
llm = ChatOpenAI(model_name="gpt-4o")
chain = prompt | llm

여기서 챗봇을 구현할 때 중요한 개념 중 하나인 ‘세션’ 에 대해서 설명해보겠습니다. 세션은 하나의 연속
된 대화를 의미합니다. 예를 들어, 한 사용자가 챗봇과 대화를 시작해서 끝낼 때까지가 하나의 세션입니
다. 세션을 사용하면 여러 사용자가 동시에 챗봇과 대화할 때 각 대화를 독립적으로 관리할 수 있습니다.
세션은 온라인 채팅에서 각각의 독립적인 대화를 구분하는 개념입니다. 예를 들어봅시다.
1. 사용자 A 가 챗봇과 대화를 시작합니다. 이것이 세션 1 입니다.
2. 동시에 사용자 B 도 같은 챗봇과 대화를 시작합니다. 이는 세션 2 가 됩니다.
3. 챗봇은 각 세션의 대화 내용을 따로 기억하고 관리합니다.이렇게 함으로써

‐ 여러 사용자가 동시에 챗봇을 사용할 수 있습니다.

‐ 챗봇은 각 사용자와의 대화 맥락을 헷갈리지 않고 유지할 수 있습니다.

‐ 한 사용자의 대화가 다른 사용자의 대화에 영향을 주지 않습니다.

코드에서 store 딕셔너리와 session_id가 이 기능을 구현하는 데 사용됩니다. store 딕셔너
리는 여러 세션의 대화들을 관리합니다. 각 세션을 독립적으로 저장하기 위해서 딕셔너리의 키는 session_id이고, 딕셔너리의 값은 해당 세션의 대화 기록입니다. 이를 통해 여러 사용자와 동시에 독립적인 대화를 관리할 수 있습니다. 여기서는 "test"라는 session_id를 생성하여 해당 세션 내에서
대화를 진행해보겠습니다.

In [ ]:
store = {}
session_id = "test"

if session_id not in store: 조건문은 현재 session_id가 store 딕셔너리에 키로 존재
하지 않는 경우를 확인합니다. 다시 말해 store 딕셔너리에 "test"라는 session_id가 아직 존재하
지 않는 경우를 확인하며, 만약 아직 존재하지 않는다면 ChatMessageHistory()는 새로운 대화 기
록 객체를 생성합니다. session_history = store[session_id]는 현재 세션의 대화 기록을
session_history 변수에 할당합니다. 새 세션이었다면 방금 생성된 빈 ChatMessageHistory
() 객체가 할당됩니다. 기존 세션이었다면 이전에 저장된 대화 기록 객체가 할당됩니다. 현재 코드가 실행되는 시점에서는 "test"라는 session_id는 신규 세션이므로 빈 ChatMessageHistory() 객체가 할당될 것입니다.

In [ ]:
if session_id not in store:
  store[session_id] = ChatMessageHistory()
session_history = store[session_id]

실제호출할때는RunnableWithMessageHistory()를사용합니다. 입력으로앞서생성한chain
, session_history, 그리고 프롬프트 템플릿에서 사용자의 입력에 사용할 변수인 "input"과 변수
인 "history"를 지정하여 모두 연결해주는 역할을 합니다.

In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: session_history,
    input_messages_key="input",
    history_messages_key="history"
)

이렇게 생성된 RunnableWithMessageHistory() 객체는 호출 시의 앞서 실습한 체인과 마찬가지
로 invoke()를 사용하여 호출합니다. 단, 해당 입력이 어떤 세션에서 사용되는지를 명시적으로 알려주
기 위해서 config={"configurable": {"session_id": "현재 질문이 어떤 세션에서 이
루어지는지를 알려주는 세션 id"}를 함께 전달해야 합니다. 챗봇에게 어디에서 만들어졌는지 질문
을 해보겠습니다.

In [ ]:
result = with_message_history.invoke(
    {"input": "당신은 어디에서 만들었습니까?"},
    config={"configurable": {"session_id": "test"}},
)
print(result.content)

안녕하세요! 저는 다양한 정보와 자료를 제공하기 위해 개발된 AI입니다. 저의 기반 기술은 여러 개발자들이 협력하여 만든 것으로, 특정한 장소에서 "제작"되었다기보다는 여러 곳에서 연구되고 발전되었습니다. 궁금한 점이 있으면 언제든지 물어보세요!


이어서 푸른 바다를 주제로 시를 지어달라는 요청을 해봅시다.

In [ ]:
result = with_message_history.invoke(
    {"input" : "푸른 바다를 주제로 감성적이고 짧은 시를 하나 지어주세요"},
    config = {"configurable": {"session_id": "test"}},
)
print(result.content)

푸른 바다의 숨결 속에  
잔잔한 파도는 속삭이고,  
반짝이는 햇살은 웃음을 짓네.  
수평선은 끝없는 꿈의 시작,  
그곳에 마음을 띄워보아요.


이번에는 석양을 주제로도 해달라는 요청을 해봅시다. 이번 요청에서는 정확하게 무엇을 해달라고는 명
시하지 않았습니다. 만약 이전 대화를 잘 기억하고 있다면 짧은 시를 지어달라는 요청임을 알고 석양을 주
제로 짧은 시를 지어줄 것입니다.

In [ ]:
result = with_message_history.invoke(
    {"input" : "석양을 주제로 짧은 시를 하나 지어주세요"},
    config = {"configurable": {"session_id": "test"}},
)
print(result.content)

석양의 붉은 노을이 물든 하늘,  
그 끝자락에 하루의 피로를 내려놓고,  
서서히 가라앉는 태양의 온기 속에  
마음은 고요히 하루를 닫는다.  
삶은 또 다른 시작을 준비하네.


정상적으로 시를 작성합니다. 앞서 확인했던 프롬프트 템플릿에서 {history}의 위치. 즉, Current
Conversation: 다음에 앞서 대화했던 내용이 누적되어 전달되었고, 과거 대화 내역을 바탕으로 석양
을 주제로 삼행시를 지어 답변해주는 것을 확인할 수 있습니다. 이번에는 각 session_id별로 대화 기
록을 저장하는 store를 출력해봅시다.

In [ ]:
store

{'test': InMemoryChatMessageHistory(messages=[HumanMessage(content='당신은 어디에서 만들었습니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 저는 다양한 정보와 자료를 제공하기 위해 개발된 AI입니다. 저의 기반 기술은 여러 개발자들이 협력하여 만든 것으로, 특정한 장소에서 "제작"되었다기보다는 여러 곳에서 연구되고 발전되었습니다. 궁금한 점이 있으면 언제든지 물어보세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 66, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44', 'id': 'chatcmpl-BxWWUQUTDE1rsmkf53q4oPu4KdVZL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--43e10007-9ad1-4583-b5ad-3bdc8cdc0f19-0', usage_metadata={'input_tokens': 66, 'output_tokens': 68, 'total_tokens': 134, 'input_token_details': {'audio': 0, 'cache_

"test"라는 key값에 앞서 대화했던 모든 기록들이 저장 되어있습니다. 사용자의 입력은 HumanMessage(content='내용')에 저장되어 있고, 그에 대한 답변은 AIMessage(content='내용')에 저장되어 있습니다.